In [22]:
!pip install fastapi uvicorn[standard] python-multipart joblib nest-asyncio
!pip install scikit-learn --upgrade

  You can safely remove it manually.


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -----------------

In [1]:
import pandas as pd
import joblib
import nest_asyncio
import uvicorn
import threading
from fastapi import FastAPI
from pydantic import BaseModel

# Creating FastAPI

In [4]:
nest_asyncio.apply()

model = joblib.load("rf_model_cleaned.pkl")
scaler = joblib.load("scaler.pkl")

df = pd.read_csv("Cleaned_and_Encoded_Cars.csv")

# Rebuild full model input feature set
full_features = model.feature_names_in_.tolist()
make_cols = [col for col in full_features if col.startswith("Make_")]
state_cols = [col for col in full_features if col.startswith("State_")]
engine_cols = [col for col in full_features if col.startswith("EngineType_")]

class CarFeatures(BaseModel):
    Rating: float
    Mileage: float
    Year: int
    Make: str
    State: str  # 2-letter state code
    EngineType: str

def build_input_vector(data: CarFeatures):
    input_data = dict.fromkeys(full_features, 0)

    input_data['Rating'] = data.Rating
    input_data['Mileage'] = data.Mileage
    input_data['Year'] = data.Year

    make_key = f"Make_{data.Make}"
    state_key = f"State_{data.State.upper()}"
    engine_key = f"EngineType_{data.EngineType}"

    if make_key in make_cols:
        input_data[make_key] = 1
    if state_key in state_cols:
        input_data[state_key] = 1
    if engine_key in engine_cols:
        input_data[engine_key] = 1

    return pd.DataFrame([input_data])[full_features]

# 🚀 Create FastAPI app
app = FastAPI(title="Vehicle Price Prediction API", version="3.0")

@app.post("/predict")
def predict_price(features: CarFeatures):
    try:
        X_input = build_input_vector(features)
        scaled_price = model.predict(X_input)
        actual_price = scaler.inverse_transform([[0, scaled_price[0], 0, 0, 0]])[0][1]  # Index 1 = Price
        return {"predicted_price": round(actual_price, 2)}
    except Exception as e:
        return {"error": str(e)}

# Running the API

In [7]:
def run_api():
    uvicorn.run(app, host="127.0.0.1", port=8003)



In [9]:
thread = threading.Thread(target=run_api)
thread.start()